In [ ]:
!pip install matplot lib
!pip install keras
!pip install tqdm
!pip3 install scikit-learn

In [1]:
from keras.callbacks import Callback
from tqdm import tqdm

class ProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']

    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch+1}/{self.epochs}")
        self.progbar = tqdm(total=self.params['steps'])

    def on_batch_end(self, batch, logs=None):
        self.progbar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        self.progbar.close()


2024-01-09 03:53:16.207611: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 03:53:16.243438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-09 03:53:16.243465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-09 03:53:16.244515: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-09 03:53:16.250073: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-09 03:53:16.250427: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
import sys

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# normalize data
def normalize_data(train, test):
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	return train_norm, test_norm

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['acc'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()


In [6]:
from keras.models import Model
from keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, Concatenate,
    Add, GlobalAveragePooling2D, Dense, AveragePooling2D
)
import numpy as np

def BasicBlock(inputs, filters, stride=1):
    x = Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    shortcut = inputs
    if stride != 1 or inputs.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same', kernel_initializer='he_normal')(inputs)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def Root(inputs, out_channels, kernel_size=1):
    x = inputs if isinstance(inputs, list) else [inputs]
    x = Concatenate(axis=-1)(x)
    x = Conv2D(out_channels, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def Tree(inputs, block, in_channels, out_channels, level=1, stride=1):
    if level == 1:
        root = Root(inputs, out_channels)
        left_node = block(inputs, out_channels, stride=stride)
        right_node = block(left_node, out_channels, stride=1)
        return root, left_node, right_node
    else:
        root = Root(inputs, out_channels * (level + 2))
        levels = []
        for i in reversed(range(1, level)):
            subtree = Tree(inputs, block, in_channels, out_channels, level=i, stride=stride)
            levels.append(subtree)
        prev_root = block(inputs, out_channels, stride=stride)
        left_node = block(prev_root, out_channels, stride=1)
        right_node = block(left_node, out_channels, stride=1)
        return root, levels, prev_root, left_node, right_node

def DLA(num_classes=10):
    random_data = np.random.randn(1, 32, 32, 3)  

    block = np.moveaxis(random_data, 1, -1)  
     
    inputs = Input(shape=(32, 32, 3))

    base = Conv2D(16, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(inputs)
    base = BatchNormalization()(base)
    base = Activation('relu')(base)

    layer1 = Conv2D(16, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(base)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)

    layer2 = Conv2D(32, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)

    layer3 = Tree(layer2, block, 32, 64, level=1, stride=1)
    layer4 = Tree(layer3[-1], block, 64, 128, level=2, stride=2)
    layer5 = Tree(layer4[-1], block, 128, 256, level=2, stride=2)
    layer6 = Tree(layer5[-1], block, 256, 512, level=1, stride=2)

    out = GlobalAveragePooling2D()(layer6[-1])
    out = Dense(num_classes, activation='softmax')(out)

    model = Model(inputs=inputs, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [8]:
# example of loading the cifar10 dataset
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint,LearningRateScheduler

trainX, trainY, testX, testY = load_dataset()
trainX, testX = normalize_data(trainX,testX)

trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

model = define_model()

epochs = 50
max_lr = 0.01
filename = 'CNN-checkpointer.keras'
batch_size = 64

#data augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(trainX, trainY, batch_size=batch_size)
steps = int(trainX.shape[0] / batch_size)

#callbacks
earlystopper = EarlyStopping(monitor='val_acc', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, 
                               verbose=1, mode='max', min_lr=0.00001)
checkpointer = ModelCheckpoint(filename, monitor='val_acc', verbose=1,
                               save_best_only=True)
def lr_schedule(epoch, lr):
        if epoch < epochs * 0.5:
            return max_lr
        else:
            return max_lr * 0.1
lr_scheduler = LearningRateScheduler(lr_schedule)

In [59]:
from keras.models import load_model

model_to_load = 'models/CNN-0.829800009727478-80epochs.keras'
model = load_model(model_to_load)

In [9]:

history = model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(valX, valY), verbose=0, callbacks=[ProgressBar(),earlystopper,reduce_lr,checkpointer,lr_scheduler])
_, acc = model.evaluate(testX, testY, verbose=0)

print('> %.3f' % (acc * 100.0))
summarize_diagnostics(history)

Epoch 1/50


  7%|▋         | 42/625 [01:05<13:29,  1.39s/it] 

KeyboardInterrupt: 

In [34]:
epochs_to_write = 20
model.save('CNN-'+str(acc)+'-'+str(epochs_to_write)+'epochs.keras')